This is a reimplementation of fastai part 2 version 3 in Swift. 
https://github.com/fastai/fastai_docs/blob/master/dev_course/dl2/02b_initializing.ipynb

In [1]:
import TensorFlow

In [2]:
public extension Tensor where Scalar : TensorFlowFloatingPoint {
    func stddev(alongAxes axes: [Int])-> Tensor<Scalar>{
        let mean = self.mean(alongAxes: axes)
        return sqrt((self - mean).squared().mean(alongAxes: axes))
    }
    
    func stddev()-> Tensor<Scalar>{
        let mean = self.mean()
        return sqrt((self - mean).squared().mean())
    }
}

In [3]:
func stats<Scalar: TensorFlowFloatingPoint>(
    _ x: Tensor<Scalar>
) -> (mean: Tensor<Scalar>, stddev: Tensor<Scalar>){
    return (mean: x.mean(), stddev: x.stddev())
}

## Why you need a good init


To understand why initialization is important in a neural net, we'll focus on the basic operation you have there: matrix multiplications. So let's just take a vector x, and a matrix a initialized randomly, then multiply them 100 times (as if we had 100 layers).

In [4]:
var x: Tensor<Float> = Tensor(randomNormal: [512, 1])
var a: Tensor<Float> = Tensor(randomNormal: [512, 512])

In [5]:
for i in 0..<100 {
    x = matmul(a, x)
}

In [6]:
stats(x)

▿ 2 elements
  - mean : nan(0x1fffff)
  - stddev : nan(0x1fffff)



The problem you'll get with that is activation explosion: very soon, your activations will go to nan. We can even ask the loop to break when that first happens:

In [7]:
var x: Tensor<Float> = Tensor(randomNormal: [512, 1])
var a: Tensor<Float> = Tensor(randomNormal: [512, 512])

In [8]:
for i in 0..<100 {
    x = matmul(a, x)
    if x.stddev().scalarized().isNaN{
        print(i)
        break
    }
}

27



It only takes around 30 multiplications! On the other hand, if you initialize your activations with a scale that is too low, then you'll get another problem:

In [9]:
var x: Tensor<Float> = Tensor(randomNormal: [512, 1])
var a: Tensor<Float> = Tensor(randomNormal: [512, 512]) * 0.01

In [10]:
for i in 0..<100 {
    x = matmul(a, x)
}

In [11]:
stats(x)

▿ 2 elements
  - mean : 0.0
  - stddev : 0.0


In [12]:
var x: Tensor<Float> = Tensor(randomNormal: [512, 1])
var a: Tensor<Float> = Tensor(randomNormal: [512, 512]) * 0.01

In [13]:
for i in 0..<100 {
    x = matmul(a, x)
    if x.stddev().scalarized().isNaN{
        print("Nan at \(i)")
        break
    }
    if x.stddev().scalarized() == 0{
        print("Zero at \(i)")
        break
    }
}

Zero at 29



Here, every activation vanished to 0. So to avoid that problem, people have come with several strategies to initialize their weight matrices, such as:

- use a standard deviation that will make sure x and Ax have exactly the same scale
- use an orthogonal matrix to initialize the weight (orthogonal matrices have the special property that they preserve the L2 norm, so x and Ax would have the same sum of squares in that case)
- use [spectral normalization](https://arxiv.org/pdf/1802.05957.pdf) on the matrix A (the spectral norm of A is the least possible number M such that torch.norm(A@x) <= M*torch.norm(x) so dividing A by this M insures you don't overflow. You can still vanish with this)

## The magic number for scaling

Here we will focus on the first one, which is the Xavier initialization. It tells us that we should use a scale equal to 1/math.sqrt(n_in) where n_in is the number of inputs of our matrix.

In [14]:
var x: Tensor<Float> = Tensor(randomNormal: [512, 1])
var a: Tensor<Float> = Tensor(randomNormal: [512, 512]) / sqrt(512)

In [15]:
for i in 0..<100 {
    x = matmul(a, x)
}

In [16]:
stats(x)

▿ 2 elements
  - mean : 4.1086645
  - stddev : 116.89491


And indeed it works. Note that this magic number isn't very far from the 0.01 we had earlier.

In [17]:
1 / sqrt(512)

0.044194173824159216


But where does it come from? It's not that mysterious if you remember the definition of the matrix multiplication. When we do y = a @ x, the coefficients of y are defined by

![image.png](imgs/math.svg)

or in code:

```y[i] = sum([c*d for c,d in zip(a[i], x)])```

Now at the very beginning, our x vector has a mean of roughly 0. and a standard deviation of roughly 1. (since we picked it that way).

In [18]:
var x: Tensor<Float> = Tensor(randomNormal: [512, 1])
stats(x)

▿ 2 elements
  - mean : -0.014157123
  - stddev : 0.9801575


NB: This is why it's extremely important to normalize your inputs in Deep Learning, the initialization rules have been designed with inputs that have a mean 0. and a standard deviation of 1.

If you need a refresher from your statistics course, the mean is the sum of all the elements divided by the number of elements (a basic average). The standard deviation shows whether the data points stay close to the mean or are far away from it. It's computed by the following formula:

![image.png](imgs/math_stddev.svg)

where m is the mean and $\sigma$ (the greek letter sigma) is the standard deviation. To avoid that square root, we also often consider a quantity called the variance, which is $\sigma$ squared.

Here we have a mean of 0, so the variance is just the mean of x squared, and the standard deviation is its square root.

If we go back to y = a @ x and assume that we chose weights for a that also have a mean of 0, we can compute the variance of y quite easily. Since it's random, and we may fall on bad numbers, we repeat the operation 100 times.

In [19]:
var totalMean: Float = 0.0
var sqr: Float = 0.0

for i in 0..<100 {
    var x: Tensor<Float> = Tensor(randomNormal: [512, 1])
    var a: Tensor<Float> = Tensor(randomNormal: [512, 512])
    var y: Tensor<Float> = matmul(a, x)
    totalMean += y.mean().scalarized()
    sqr += pow(y, 2).mean().scalarized()
}

In [20]:
print(totalMean/100, sqr/100)

-0.08908906 522.89276


Now that looks very close to the dimension of our matrix 512. And that's no coincidence! When you compute y, you sum 512 product of one element of a by one element of x. So what's the mean and the standard deviation of such a product of one element of a by one element of x? We can show mathematically that as long as the elements in a and the elements in x are independent, the mean is 0 and the std is 1.

This can also be seen experimentally:

In [21]:
var totalMean: Float = 0.0
var sqr: Float = 0.0

for i in 0..<10000 {
    var x: Tensor<Float> = Tensor(randomNormal: [1])
    var a: Tensor<Float> = Tensor(randomNormal: [1])
    var y: Tensor<Float> = a * x
    totalMean += y.mean().scalarized()
    sqr += pow(y, 2).scalarized()
}

In [22]:
print(totalMean/10000, sqrt(sqr/10000))

0.01424955 1.0051857


## Adding ReLU in the mix

We can reproduce the previous experiment with a ReLU, to see that this time, the mean shifts and the variance becomes 0.5. This time the magic number will be math.sqrt(2/512) to properly scale the weights of the matrix.

In [23]:
var totalMean: Float = 0.0
var sqr: Float = 0.0

for i in 0..<10000 {
    var x: Tensor<Float> = Tensor(randomNormal: [1])
    var a: Tensor<Float> = Tensor(randomNormal: [1])
    var y: Tensor<Float> = a * x
    y = max(y, 0)
    totalMean += y.mean().scalarized()
    sqr += pow(y, 2).scalarized()
}

In [24]:
print(totalMean/10000, sqr/10000)

0.3242501 0.5527525


We can double check by running the experiment on the whole matrix product. The variance becomes 512/2 this time:

In [25]:
var totalMean: Float = 0.0
var sqr: Float = 0.0

for i in 0..<100 {
    var x: Tensor<Float> = Tensor(randomNormal: [512, 1])
    var a: Tensor<Float> = Tensor(randomNormal: [512, 512])
    var y: Tensor<Float> = matmul(a, x)
    y = max(y, 0)
    totalMean += y.mean().scalarized()
    sqr += pow(y, 2).mean().scalarized()
}

In [26]:
print(totalMean/100, sqr/100, 512/2)

9.045997 257.05624 256



Or that scaling the coefficient with the magic number gives us a scale of 1.

In [27]:
var totalMean: Float = 0.0
var sqr: Float = 0.0

for i in 0..<100 {
    var x: Tensor<Float> = Tensor(randomNormal: [512, 1])
    var a: Tensor<Float> = Tensor(randomNormal: [512, 512]) * sqrt(2/512)
    var y: Tensor<Float> = matmul(a, x)
    y = max(y, 0)
    totalMean += y.mean().scalarized()
    sqr += pow(y, 2).mean().scalarized()
}

In [28]:
print(totalMean/100, sqr/100)

0.5600105 0.9899393


The math behind is a tiny bit more complex, and you can find everything in the [Kaiming](https://arxiv.org/abs/1502.01852) and the [Xavier](http://proceedings.mlr.press/v9/glorot10a.html) paper but this gives the intuition behind those results.